In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from skimage.feature import hog
from skimage import exposure
import os
import cv2
from skimage import io, color, feature, exposure
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import silhouette_score
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score  
from sklearn.metrics import recall_score 
from sklearn.metrics import precision_score 
from sklearn.metrics import precision_recall_curve 
from sklearn.metrics import roc_curve , auc , confusion_matrix ,roc_auc_score 
from IPython.display import Image

In [ ]:
# Function to extract HOG features from an image and visualize it
def extract_hog_features(image):
    # Convert the image to grayscale
    gray_image = color.rgb2gray(image)

    # Calculate HOG features
    hog_features, hog_image = feature.hog(gray_image, visualize=True)

    # Enhance the contrast of the HOG image for better visualization
    hog_image_rescaled = exposure.rescale_intensity(hog_image, in_range=(0, 10))

    return hog_features, hog_image_rescaled

In [ ]:
# Path to the root folder of your dataset
class_folder = "D:/Level 3/ML/Project/Image_Dataset"

# List all subdirectories (assuming each subdirectory corresponds to a class)
features_list=[]
labels_list=[]
age = []
gender = []
race = []

for image_filename in os.listdir(class_folder):
    image_path = os.path.join(class_folder, image_filename)
    
    #append age and geder and race for each image
    p = (image_filename.split('.')[0]).split('_')
    age.append(p[0])
    gender.append(p[1])
    race.append(p[2])

    # Load the image
    image = io.imread(image_path)
    #resize image into same saize 64*128
    image = cv2.resize(image, (64,64))

    # Extract HOG features and visualize
    hog_features, hog_image = extract_hog_features(image)

    # Display the original image and the HOG features
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(2, 2), sharex=True, sharey=True)

    ax1.axis('off')
    ax1.imshow(image, cmap=plt.cm.gray)

    ax2.axis('off')
    ax2.imshow(hog_image, cmap=plt.cm.gray)

    plt.show()

    # Append HOG features to the features list
    features_list.append(hog_features)

    #not important
    # Append the label to the labels list
    labels_list.append(class_folder)



In [ ]:
# Convert lists to NumPy arrays
features_array = np.array(features_list)
labels_array = np.array(labels_list)

#not important
# Use LabelEncoder to convert class names into numeric labels
label_encoder = LabelEncoder()
numeric_labels = label_encoder.fit_transform(labels_array)

print("\n*********************\n")
print(len(features_array))


In [ ]:
# Convert 'age' list to numpy array
age_array = np.array(age).reshape(-1, 1)  # Reshape as a column vector

# Convert 'gender' list to numpy array
gender_array = np.array(gender).reshape(-1, 1)  # Reshape as a column vector

# Convert 'race' list to numpy array
race_array = np.array(race).reshape(-1, 1)  # Reshape as a column vector

# Create the dataset by horizontally stacking age_array with features_array
dataset = np.hstack((age_array,gender_array, race_array,features_array))

# Convert dataset to a pandas DataFrame
column_names = ['Age']+['Gender'] +['Race'] + [f'Feature_{i}' for i in range(features_array.shape[1])]  # Creating column names
df = pd.DataFrame(dataset, columns=column_names)

In [ ]:
# save dataset in my drive 
df.to_csv('AgeWithoutSum.csv', index=False)

In [ ]:
#load data from my devise 
dataset = pd.read_csv('AgeWithoutSum.csv')


In [ ]:
dataset.head(5)

In [ ]:
dataset.shape

In [ ]:
# normalization agea 
dataset['Age'] = MinMaxScaler(feature_range=(0,1)).fit_transform(dataset[['Age']])

In [ ]:
dataset.head(5)

In [ ]:
#spliot into featurs and target 
data_x = dataset.drop(dataset.columns[1], axis=1)  #All Features expect gender 
data_y = dataset.iloc[:, 1]   #Target only gender

In [ ]:
data_x.head(5)

In [ ]:
data_y.head(5)

In [ ]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data_x, data_y, test_size=0.2, random_state=42)

In [ ]:
print("X_train Shape :",X_train.shape)
print("X_test Shape :",X_test.shape)
print("y_train Shape :",y_train.shape)
print("y_test Shape :",y_test.shape)

# Logistic Regression Model

In [ ]:
# Logistic Regression
lr = LogisticRegression(penalty='l2',solver='saga',C=1.0,random_state=11)

In [ ]:
lr.fit(X_train, y_train)

In [ ]:
# prediction
y_pred = lr.predict(X_test)

In [ ]:
# prediction
y_pred = lr.predict(X_test)
# test and train score 
print("(Logistic Regression)Train Score :",lr.score(X_train,y_train)) 
print("(Logistic Regression)Teast Score :",lr.score(X_test,y_test))

In [ ]:
# classes
print("Classes is : ",lr.classes_)

In [ ]:
# Itreations 
print("Itreations is : ",lr.n_iter_)

In [ ]:
#  intercept_ 
print("Intercept is : ",lr.intercept_)

In [ ]:
#   coef_
print("coef is : ",lr.coef_)

In [ ]:
# Get parameters for this estimator.
print("Prediction  : ",lr.predict(X_test)[:10])

In [ ]:
#  Predict class labels for samples in X.
print("Prediction Probabilities : ",lr.predict_proba(X_test)[:2])

# Metrics

In [ ]:
#Calculating Confusion Matrix 
cm = confusion_matrix(y_test,y_pred)
print("Confusion Matrix :")
print(cm)

In [ ]:
# plot confusion matrix 
sns.heatmap(cm, center = True) 
plt.show() 

In [ ]:
#Calculating Accuracy Score 
acScore = accuracy_score(y_test,y_pred,normalize=False)
print("Accuracy Score : ",acScore)

In [ ]:
#Calculating Accuracy Score 
acScore = accuracy_score(y_test,y_pred)
print("Accuracy Score : ",acScore)

In [ ]:
#Calculating ROC AUC Score:
ROCAUCScore = roc_auc_score(y_test,y_pred, average='micro') 
print('ROCAUC Score : ', ROCAUCScore) 

In [ ]:
# ROC curve 
# Predict probabilities on the test set
y_probs = lr.predict_proba(X_test)[:, 1]

# Calculate the ROC curve
fpr, tpr, thresholds = roc_curve(y_test, y_probs)
roc_auc = auc(fpr, tpr)

# Plotting the ROC curve
plt.figure()
plt.plot(fpr, tpr, color='blue', lw=2, label='ROC curve (AUC = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='red', linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc="lower right")
plt.show()

In [ ]:
#Loss 
#plt.plot(lr.losscurve,label="train")
#lr.fit(X_train,y_train)                    #doubt
#plt.plot(lr.losscurve,label="validation") #doubt
#plt.legend()